In [288]:
import matplotlib.pyplot as plt 
import matplotlib.gridspec as GridSpec
import matplotlib
import pandas as pd 
import numpy as np 
import os
import shutil

In [289]:
class folder_process():
    # инициализация, проверка существования папки, проверка пустоты папки
    def __init__(self, main_folder: str) -> None:
        if not os.path.exists(main_folder):
            raise FileNotFoundError(f'directory \'{main_folder}\' is not exist')
        elif  (len(os.listdir(main_folder)) == 0):
            raise ValueError(f'directory \'{main_folder}\' is empty')
        else:
            self.main_folder = main_folder
            self.list_of_subdirs = os.listdir(main_folder)
            self.main_file_path = os.getcwd()
            
    # поиск всех вложенных директорий в основной папке
    def _find_subdirectories(self) -> None:
        self.subdirs_list = set()
        for i in self.list_of_subdirs:
            if os.path.isdir('\\'.join([self.main_folder, i])):
                self.subdirs_list.add(f'{i}')

    # составление списка header файлов в основной папке
    def _find_headers(self) -> None:
        self.headers_list = set()
        for catalog in self.list_of_subdirs:
            splited_catalog = catalog.split('.')[-1]
            if splited_catalog == 'header':
                self.headers_list.add(catalog.replace('.header', ''))

    # считывание данных об измерениях из файлов во вложенных директориях
    def _measurments_in_subdirectories(self) -> None:
        self._find_subdirectories()
        if len(self.subdirs_list) == 0:
            raise ValueError(f'directory \'{self.main_folder}\' does not contain subdirectories')
        self.all_measurs_from_subdirs = {}
        for i in self.subdirs_list:
            subdir_catalogs_list = [i.replace('.data', '') for i in os.listdir('\\'.join([self.main_folder, i]))]
            self.all_measurs_from_subdirs[i] = subdir_catalogs_list
        
    # считывание данных об измерениях из header файлов
    def _measurments_in_headers(self)-> None:
        self._find_headers()
        if len(self.headers_list) == 0:
            raise ValueError(f'directory \'{self.main_folder}\' does not contain header\'s files')
        splite_line = '______________________________________________________'
        self.all_measurs_from_headers = {}
        self.all_types_from_headers = {}
        for i in self.headers_list:
            self.all_measurs_from_headers[i] = set()
            self.all_types_from_headers[i] = {}
            with open('\\'.join([self.main_folder, i + '.header'])) as file:
                header_lines = []
                for k in file.readlines()[:-2]:
                    header_lines.append(k.replace('\n', ''))
            for j in range(len(header_lines)):
                if header_lines[j] == splite_line:
                    measurment_number, *measurmet_type = header_lines[j + 3].split()
                    self.all_measurs_from_headers[i].add(measurment_number)
                    self.all_types_from_headers[i][measurment_number] = ' '.join(measurmet_type)
                else:
                    continue

    # поиск пересечений в данных об измерениях
    def get_dict_of_measurments(self)-> None:

        subdirectories = list(self.headers_list & self.subdirs_list)
        self.dict_of_measurments = {}
        for i in subdirectories:
            self.dict_of_measurments[i] = sorted(list(self.all_measurs_from_headers[i] & set(self.all_measurs_from_subdirs[i])))
        return self.dict_of_measurments


In [290]:
main = folder_process('hBN_1_2')
main.measurments_in_subdirectories()
main.measurments_in_headers()
main.get_dict_of_measurments()
main.dict_of_measurments

{'16_box': ['1',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '2',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '3',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '4',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '5',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '6',
  '60',
  '7',
  '8',
  '9'],
 'box_test': ['1', '2', '3', '4'],
 '17': ['1',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '2',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '3',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '4',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '5',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '6',
  '60',
  '61',
  '62',
  '63',
  '